In [ ]:
from lxml import html
import requests

headers = {"user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
                         "(KHTML, like Gecko) Ubuntu Chromium/71.0.3578.80 "
                         "Chrome/71.0.3578.80 Safari/537.36"}

url = "https://letterboxd.com/film/spider-man-across-the-spider-verse/"
page = requests.get(url, headers=headers)
tree = html.fromstring(page.content)

x = tree.xpath('//span[@id="cast-overflow"]/a/text()')
print(x)

In [ ]:
from lxml import html
from concurrent.futures import ThreadPoolExecutor
import requests
import pandas as pd

def fetch(url):
    response = requests.get(url)
    return response.text

def request(urls):
    with ThreadPoolExecutor() as executor:
        results = executor.map(fetch, urls)
    return results

headers = {"user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
                         "(KHTML, like Gecko) Ubuntu Chromium/71.0.3578.80 "
                         "Chrome/71.0.3578.80 Safari/537.36"}

# name = input("Introduce tu nombre de Letterbox:")
name = "elviragalera"
url = f"https://letterboxd.com/{name}/films/"
page = requests.get(url, headers=headers)
tree = html.fromstring(page.content)

pages = tree.xpath('/html/body/div[1]/div/div/section/div[2]/div[3]/ul/li/a')[-1].text

page_urls = []
for i in range(1, int(pages) + 1):
    page_urls += [f"https://letterboxd.com/{name}/films/page/" + str(i)]

rs1 = request(page_urls)

directors = {}
film_urls = []

for html_content in rs1:
    tree = html.fromstring(html_content)
    result = tree.xpath('/html/body/div[1]/div/div/section/ul/li/div')
    for film in result:
        link = "https://letterboxd.com" + film.attrib.get('data-target-link')
        film_urls += [link]

rs = request(film_urls)
directors = {}
actors = {}

for html_content in rs:
    tree = html.fromstring(html_content)
    director = tree.xpath('//div[@id="tabbed-content"]/div[@id="tab-crew"]/div[1]/p/a/text()')
    for d in director:
        directors[d] = directors.get(d, 0) + 1
    actor = tree.xpath('//div[@id="tab-cast"]/div/p/a/text()')
    actors_overflow = tree.xpath('//span[@id="cast-overflow"]')
    if actors_overflow:
        actor += tree.xpath('//span[@id="cast-overflow"]/a/text()')
    for a in actor:
        actors[a] = actors.get(a, 0) + 1

df_directors = pd.DataFrame(directors.items(), columns=['Director', 'Times'])
df_actors = pd.DataFrame(actors.items(), columns=["Actor", "Times"])
df_actors = df_actors[df_actors['Actor'] != "Show All…"]

print(df_directors.sort_values(by="Times", ascending=False))
print(df_actors.sort_values(by="Times", ascending=False))

In [96]:
df_actors.sort_values(by="Times", ascending=False)[1:20]

,Actor,Times
1250,Chris Evans,13
2617,Scarlett Johansson,10
113,Chris Pratt,9
119,Bradley Cooper,8
2866,Samuel L. Jackson,8
115,Dave Bautista,8
114,Zoe Saldaña,8
1133,Chris Hemsworth,7
2847,Mark Ruffalo,7
2846,Robert Downey Jr.,7


c:\Users\javip\AppData\Local\Programs\Python\Python39\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:
